In [55]:
import os, sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))
import database
from snowballing.operations import reload, work_by_varname
from snowballing.snowballing import snowballing, create_provenance, log_to_provn
reload()


In [59]:

reload()
frontier = {work_by_varname(x) for x in (
    "wohlin2014a",
)}
filter_function = lambda x: x.category in ("snowball", "forward", "backward")
frontier, log, visited = create_provenance(
    frontier, filter_function, backward_first=True
)

log

[Step(name='start', new_references=1, new_related=1, total_visited=1, total_related=1),
 Step(name='backward', new_references=21, new_related=1, total_visited=22, total_related=2),
 Step(name='forward', new_references=42, new_related=0, total_visited=64, total_related=2),
 Step(name='backward', new_references=0, new_related=0, total_visited=64, total_related=2)]

In [69]:
from copy import copy

from snowballing.models import Work

def aplica_par_backward_forward(initial_set, filter_function=None):
    frontier = copy(initial_set)
    with snowballing(frontier, filter_function) as (backward, forward, log, visited):
        order = [backward, forward]
        new_frontier = frontier
        count = 0
        print("operacao, seed set, encontrado")
        while count < 2:
            temp = set()
            for func in order:
                count = count + 1 if not new_frontier else 0
                new_frontier = func(frontier)
                print(func.__name__, [x.metakey for x in frontier], [x.metakey for x in new_frontier])
                temp.update(new_frontier)
            frontier.update(temp)    
            
    return frontier, log, visited

reload()
frontier = {work_by_varname(x) for x in (
    "wohlin2014a",
)}
filter_function = lambda x: x.category in ("snowball", "forward", "backward")
frontier, log, visited = aplica_par_backward_forward(
    frontier, filter_function,
)

log

operacao, seed set, encontrado
backward ['wohlin2014a'] ['kitchenham2004a']
backward ['wohlin2014a', 'kitchenham2004a'] []
backward ['wohlin2014a', 'kitchenham2004a'] []
backward ['wohlin2014a', 'kitchenham2004a'] []


[Step(name='start', new_references=1, new_related=1, total_visited=1, total_related=1),
 Step(name='backward', new_references=21, new_related=1, total_visited=22, total_related=2),
 Step(name='backward', new_references=0, new_related=0, total_visited=22, total_related=2),
 Step(name='backward', new_references=0, new_related=0, total_visited=22, total_related=2),
 Step(name='backward', new_references=0, new_related=0, total_visited=22, total_related=2)]

In [71]:
from copy import copy

from snowballing.models import Work

gbackward = None

def todo_backward_e_depois_todo_forward(initial_set, filter_function=None):
    frontier = copy(initial_set)
    with snowballing(frontier, filter_function) as (backward, forward, log, visited):
        
        print("operacao, seed set, encontrado")
        new_frontier = frontier
        global gbackward
        gbackward = backward
        while new_frontier:
            new_frontier = backward(frontier)
            print("backward", [x.metakey for x in frontier], [x.metakey for x in new_frontier])
            frontier.update(new_frontier)
        new_frontier = frontier
        while new_frontier:
            new_frontier = forward(frontier)
            print("forward", [x.metakey for x in frontier], [x.metakey for x in new_frontier])
            frontier.update(new_frontier)
        
    return frontier, log, visited

reload()
frontier = {work_by_varname(x) for x in (
    "wohlin2014a",
)}
filter_function = lambda x: x.category in ("snowball", "forward", "backward")
frontier, log, visited = todo_backward_e_depois_todo_forward(
    frontier, filter_function,
)

log
#len(visited)

operacao, seed set, encontrado
forward ['wohlin2014a'] []


[Step(name='start', new_references=1, new_related=1, total_visited=1, total_related=1),
 Step(name='forward', new_references=42, new_related=0, total_visited=43, total_related=1)]

In [68]:
snowballing??

In [26]:
log

[Step(name='start', new_references=1, new_related=1, total_visited=1, total_related=1),
 Step(name='backward', new_references=21, new_related=1, total_visited=22, total_related=2),
 Step(name='forward', new_references=42, new_related=0, total_visited=64, total_related=2),
 Step(name='backward', new_references=0, new_related=0, total_visited=64, total_related=2)]

In [13]:
frontier

{Guidelines for Snowballing in Systematic Literature Studies and a Replication in Software Engineering,
 Evidence-based software engineering}

In [63]:
create_provenance??

In [65]:
print(log_to_provn(log))

document
  default <http://example.org/>

  activity(start, -, -)
  entity(s0, [type="Set", length="1"])
  wasGeneratedBy(g0; s0, start, -)

  activity(backward1, -, -, [found="21", related="1"])
  used(u0; backward1, s0, -)
  entity(s1, [type="Set", length="2"])
  wasGeneratedBy(g1; s1, backward1, -)
  wasDerivedFrom(s1, s0, backward1, g1, u0, [prov:type="prov:Revision"])

  activity(forward1, -, -, [found="42", related="0"])
  used(u1; forward1, s1, -)

  activity(backward2, -, -, [found="0", related="0"])
  used(u2; backward2, s1, -)

  activity(forward2, -, -, [found="0", related="0"])
  used(u3; forward2, s1, -)

endDocument


In [ ]:
with open("output/snowballing.provn", "w") as provn:
    provn.write(log_to_provn(log))

In [ ]:
!provconvert.bat -infile snowballing.provn -outfile snowballing.dot

In [ ]:
with open("output/snowballing.dot", "r") as prov_dot:
    text = prov_dot.read().replace('rankdir="BT";', 'rankdir="RL"; ranksep="0.1";')
with open("output/snowballing.dot", "w") as prov_dot:
    prov_dot.write(text)

In [ ]:
!dot -Tpng output/snowballing.dot -ooutput/snowballing.png
!dot -Tsvg output/snowballing.dot -ooutput/snowballing.svg
!inkscape -D -z --file=output/snowballing.svg --export-pdf=../csur/figs/snowballing.pdf

In [ ]:
len(visited)

In [ ]:
len([x for x in visited if x.category in {"snowball"}])

In [ ]:
from snowballing.approaches import get_approaches
len(get_approaches())

In [ ]:
len([(x, x.year) for x in visited
     if x.category in {"nofile"}
     if x.place.name != "Tech Report"
     if x.place.name != "Book"
     if x.place.name != "Patent"
     if x.display != x.place.acronym
    ])

In [ ]:
len([x for x in visited if x.category in {"lang"}])

In [ ]:
len([x for x in visited if x.category in {"site"}])

In [ ]:
len([x for x in visited
     if x.place.name == "Tech Report"
    ])

In [ ]:
len([x for x in visited
     if x.place.name == "Book"
    ])

In [ ]:
{x.category for x in visited}

In [ ]:
[(x.year, x.authors, x.name, x.place.name, x.request) for x in visited
 if x.category in {"nofile"}
 if x.place.name != "Tech Report"
 if x.place.name != "Book"
 if x.place.name != "Patent"
 if x.display != x.place.acronym]

In [ ]:
len(_)